# Incremental Ernollment Trial with voxceleb1
---------

trial 길이가 충분히 긴 set을 만들고자 한다.

## Environment

In [1]:
%load_ext autoreload
%autoreload 2
%pylab
%matplotlib inline

import os
import sys
import pickle
import pandas as pd
from utils import key2df

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


## Dataframes & embeddings

In [38]:
voxc1_si = pd.read_csv("/dataset/SV_sets/voxceleb1/dataframes/voxc1_si_with_frames.csv", index_col=False)
voxc1_sv = pd.read_csv("/dataset/SV_sets/voxceleb1/dataframes/voxc1_sv.csv")

In [39]:
voxc1_df = pd.concat([voxc1_si, voxc1_sv], sort=True)

In [40]:
spk_uttr_stat = voxc1_df.spk.value_counts()

## Held-out validation set

utterance가 150개 넘는 것들을 trial spks로 빼고 나머지는 validation spk로 빼었다.  
그리고 validation spk 음성을 이용해서 threshold를 정하기 위한 trial을 만든다.

In [41]:
dev_spks = spk_uttr_stat[spk_uttr_stat <= 150].index.tolist()
dev_uttrs = voxc1_df[voxc1_df.spk.isin(dev_spks)]
eval_spks = spk_uttr_stat[spk_uttr_stat > 150].index.tolist()
eval_uttrs = voxc1_df[voxc1_df.spk.isin(eval_spks)]

In [42]:
voxc1_dev_df = voxc1_df[voxc1_df.spk.isin(dev_spks)]
voxc1_eval_df = voxc1_df[voxc1_df.spk.isin(eval_spks)]

In [83]:
voxc1_dev_df['label'] = voxc1_dev_df.groupby('spk').ngroup().values

/opt/conda/envs/pytorch-py3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [ ]:
pd.DataFrame.reset_index()

In [90]:
voxc1_dev_df = voxc1_dev_df.sort_values('id').reset_index(drop=True)

In [94]:
voxc1_dev_df = voxc1_dev_df.fillna('train')

In [97]:
voxc1_dev_df.to_csv("/dataset/SV_sets/voxceleb1/dataframes/voxc1_dev.csv", index=False)

In [98]:
voxc1_eval_df.to_csv("/dataset/SV_sets/voxceleb1/dataframes/voxc1_eval.csv", index=False)

In [99]:
len(dev_spks)

945

## Design trials for each enr_spk

In [150]:
print(f"number of spks: {len(eval_spks)}")

number of spks: 306


In [148]:
trials = []

for enr_spk in eval_spks[:2]: # TODO: multiple enrolled speakers case
    target_uttrs = voxc1_eval_df[voxc1_eval_df.spk == enr_spk]
    nonTarget_uttrs = voxc1_eval_df[voxc1_eval_df.spk != enr_spk]
    target_uttrs.loc[:, 'label'] = 1
    nonTarget_uttrs.loc[:, 'label'] = 0
    
    enr_uttrs =  target_uttrs[(target_uttrs.frames > 300) & (target_uttrs.frames < 400)]
    for idx, enr_uttr in enr_uttrs.iterrows():
        target_uttrs_ = target_uttrs.drop(index=idx)
        # adapt trials
        adapt_target_uttrs = target_uttrs_.sample(n=50)
        adapt_nonTarget_uttrs = nonTarget_uttrs.sample(n=200)

        # test trials
        test_target_uttrs = target_uttrs_.drop(index=adapt_target_uttrs.index)
        test_nonTarget_uttrs = nonTarget_uttrs.drop(index=adapt_nonTarget_uttrs.index).sample(n=3000)

        # shuffle trials and it will be fixed for consistency
        adapt_trial = pd.concat([adapt_target_uttrs, adapt_nonTarget_uttrs]).sample(frac=1)
        test_trial = pd.concat([test_target_uttrs, test_nonTarget_uttrs]).sample(frac=1)

        trials += [[enr_spk, 
                   [enr_uttr.id], 
                   (np.array(adapt_trial.id), np.array(adapt_trial.label)),
                   (np.array(test_trial.id),np.array(test_trial.label)),
                 ]]

/opt/conda/envs/pytorch-py3.6/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


## Save the trials

In [149]:
save_dir = "./trials/enr306/short_enr_test/"
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
pickle.dump(trials, open(save_dir + "/trials.pkl", "wb"))    